In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/march-machine-learning-mania-2024/Conferences.csv
/kaggle/input/march-machine-learning-mania-2024/sample_submission.csv
/kaggle/input/march-machine-learning-mania-2024/WNCAATourneyDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2024/WRegularSeasonCompactResults.csv
/kaggle/input/march-machine-learning-mania-2024/MNCAATourneySeedRoundSlots.csv
/kaggle/input/march-machine-learning-mania-2024/MRegularSeasonDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2024/MNCAATourneyCompactResults.csv
/kaggle/input/march-machine-learning-mania-2024/MGameCities.csv
/kaggle/input/march-machine-learning-mania-2024/WGameCities.csv
/kaggle/input/march-machine-learning-mania-2024/MSeasons.csv
/kaggle/input/march-machine-learning-mania-2024/WNCAATourneySlots.csv
/kaggle/input/march-machine-learning-mania-2024/MSecondaryTourneyTeams.csv
/kaggle/input/march-machine-learning-mania-2024/2024_tourney_seeds.csv
/kaggle/input/march-machine-learning-mania-2024/Cities.csv
/

In [2]:
!pip install -q jupyter-black
!pip install -q --upgrade xgboost[dask]

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.19.1 requires aiohttp, which is not installed.
fury 0.10.0 requires aiohttp>=3.8.4, which is not installed.
jupyter-server-proxy 4.1.0 requires aiohttp, which is not installed.
keras-cv 0.9.0 requires keras-core, which is not installed.
keras-nlp 0.12.1 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.0 which is incompatible.
jupyterlab 4.2.1 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.0 requires 

In [3]:
import jupyter_black

jupyter_black.load()

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import matplotlib as mpl
from matplotlib.patches import Circle, Rectangle, Arc
import seaborn as sns

from sklearn.metrics import accuracy_score, log_loss
import xgboost as xgb
from sklearn.model_selection import GroupKFold

plt.style.use("fivethirtyeight")
mypal = plt.rcParams["axes.prop_cycle"].by_key()["color"]

In [5]:
!ls -GFlash ../input/march-machine-learning-mania-2024/

total 144M
   0 drwxr-xr-x 2 nobody    0 May 13 11:16 ./
4.0K drwxr-xr-x 3 root   4.0K Jun  5 15:37 ../
4.0K -rw-r--r-- 1 nobody 1.4K May 13 11:16 2024_tourney_seeds.csv
 12K -rw-r--r-- 1 nobody 9.1K May 13 11:16 Cities.csv
4.0K -rw-r--r-- 1 nobody 1.7K May 13 11:16 Conferences.csv
168K -rw-r--r-- 1 nobody 168K May 13 11:16 MConferenceTourneyGames.csv
2.5M -rw-r--r-- 1 nobody 2.5M May 13 11:16 MGameCities.csv
111M -rw-r--r-- 1 nobody 111M May 13 11:16 MMasseyOrdinals_thruSeason2024_day128.csv
 72K -rw-r--r-- 1 nobody  72K May 13 11:16 MNCAATourneyCompactResults.csv
132K -rw-r--r-- 1 nobody 129K May 13 11:16 MNCAATourneyDetailedResults.csv
 16K -rw-r--r-- 1 nobody  16K May 13 11:16 MNCAATourneySeedRoundSlots.csv
 40K -rw-r--r-- 1 nobody  38K May 13 11:16 MNCAATourneySeeds.csv
 52K -rw-r--r-- 1 nobody  50K May 13 11:16 MNCAATourneySlots.csv
5.3M -rw-r--r-- 1 nobody 5.3M May 13 11:16 MRegularSeasonCompactResults.csv
 11M -rw-r--r-- 1 nobody  11M May 13 11:16 MRegularSeasonDetailedResults.

**Files we are interested in:**

* ***MRegularSeasonCompactResults.csv & WMRegularSeasonCompactResults.csv**

    All game results from the regular season.
* **MNCAATourneyCompactResults.csv & WNCAATourneyCompactResults.csv**

    All game results from past tournaments.
* **MNCAATourneySeeds.csv & MNCAATourneySeeds.csv**

     The seeding for the tournaments
* *** 2024_tourney_seeds.csv**

   File that will be updated with 2024 seeds once released (2023 seeds prior to that)

In [6]:
DATA_PATH = "../input/march-machine-learning-mania-2024/"

In [7]:
df_seeds = pd.concat(
    [
        pd.read_csv(DATA_PATH + "MNCAATourneySeeds.csv").assign(League="M"),
        pd.read_csv(DATA_PATH + "WNCAATourneySeeds.csv").assign(League="W"),
    ],
).reset_index(drop=True)

df_season_results = pd.concat(
    [
        pd.read_csv(DATA_PATH + "MRegularSeasonCompactResults.csv").assign(League="M"),
        pd.read_csv(DATA_PATH + "WRegularSeasonCompactResults.csv").assign(League="W"),
    ]
).reset_index(drop=True)

df_tourney_results = pd.concat(
    [
        pd.read_csv(DATA_PATH + "MNCAATourneyCompactResults.csv").assign(League="M"),
        pd.read_csv(DATA_PATH + "WNCAATourneyCompactResults.csv").assign(League="W"),
    ]
).reset_index(drop=True)